In [1]:
import nest_asyncio # async
nest_asyncio.apply()

from dotenv import load_dotenv # for .env file
import os
load_dotenv() # load .env file

# Structures for the bot
from Structures.stack import Stack
from Structures.queue import Queue

history = Stack('history:') # history of commands
queue = Queue('queue:') # queue of users to access the bot

# discord functions
import discord 
from discord.ext import commands

intents = discord.Intents.all() # Bot intents (permissions)
client = commands.Bot(command_prefix="/", intents = intents) # Bot prefix

In [2]:
# QUEUE SETUP
async def manage_queue():
    await client.wait_until_ready() # wait until the bot is ready
    while not client.is_closed(): # while the bot is running
        if not queue.isEmpty(): # if the queue is not empty
            user = queue.dequeue() # get the first user in the queue
            await user
        await asyncio.sleep(5)
        
async def add_to_queue(user):
    queue.enqueue(user)
    await user.send("You have been added to the queue. Please wait for your turn.")

In [3]:
@client.command(name="del")
async def delete(ctx):
    if queue.isEmpty():
        history.push("delete")
        await ctx.channel.purge(limit=10)
    else:
        await add_to_queue(ctx.author)
    
@client.command(name="history_head")
async def print_history_head(ctx):
    await ctx.channel.send(history.peek())
    
@client.command(name="history")
async def print_history_next(ctx):
    await ctx.channel.send(history.print())
    
@client.command(name="history_clear") # NEEDS TEST
async def clear_history(ctx):
    await ctx.channel.send(history.clear())
    
@client.command(name="history_index") # NEEDS TEST
async def print_history_index(ctx):
    await ctx.channel.send("Please enter your input:")
    def check(m):
        return m.author == ctx.author and m.channel == ctx.channel
    msg = await client.wait_for("message", check=check)
    if msg.content.isdigit():
        await ctx.channel.send(history.print_index(msg.content))
    
@client.command(name="commands")
async def commands(ctx):
    await ctx.channel.send("Available commands: \nhistory: get logs of server\nclear-history: clear commands history\nhistory_head: print first command of history\nhistory_index")

@client.event
async def on_ready():
    print("Le bot est prêt !")

@client.event
async def on_typing(channel, user, when):
     await channel.send(user.name+" is typing")

@client.event
async def on_member_join(member):
    general_channel = client.get_channel(977137496720826368)
    await general_channel.send("Bienvenue sur le serveur ! "+ member.name)

@client.event
async def on_message(message):
  if message.author == client.user:
    return   

  if message.content.startswith("hello"):
    await message.channel.send("hello")

  await client.process_commands(message)

SyntaxError: invalid syntax (2331227265.py, line 8)

In [ ]:
client.run(os.getenv('BOT_TOKEN'))